In [ ]:
# !pip install datasets
# !pip install accelerate -U

In [ ]:
from datasets import load_dataset
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load the dataset
dataset = load_dataset('GaryHuang/NTU_geolocation')
full_dataset = dataset['train']

# Convert string labels to numeric labels
label_encoder = LabelEncoder()
numerical_labels = label_encoder.fit_transform(full_dataset['label'])  # Replace 'label' with the actual label column name if different

# Replace the original labels with numeric labels
full_dataset = full_dataset.remove_columns('label')
full_dataset = full_dataset.add_column('label', numerical_labels)

# Define the sample size (e.g., 10% of the full dataset)
sample_size = int(1 * len(full_dataset))  # Adjust this percentage based on your needs

# Shuffle the dataset and sample a subset
np.random.seed(42)  # For reproducibility
sample_indices = np.random.permutation(len(full_dataset))[:sample_size]
sampled_dataset = full_dataset.select(sample_indices)

# Define the sizes for train, validation, and test split: 80%, 10%, 10%
train_size = int(0.8 * len(sampled_dataset))
validation_size = int(0.1 * len(sampled_dataset))
test_size = len(sampled_dataset) - train_size - validation_size  # Ensure all samples are used

# Generate shuffled indices for the sampled dataset
indices = np.random.permutation(len(sampled_dataset))

# Split indices for each dataset
train_indices = indices[:train_size]
validation_indices = indices[train_size:train_size + validation_size]
test_indices = indices[train_size + validation_size:]

# Create subsets using the indices
train_subset = sampled_dataset.select(train_indices)
validation_subset = sampled_dataset.select(validation_indices)
test_subset = sampled_dataset.select(test_indices)

# Print the sizes to confirm
print(f"Train subset size: {len(train_subset)}")
print(f"Validation subset size: {len(validation_subset)}")
print(f"Test subset size: {len(test_subset)}")

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image

data_transforms = {
    'train': transforms.Compose([
        transforms.RandAugment(num_ops = 5, magnitude = 10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = Image.fromarray(np.array(item['image']))
        label = item['label']
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def create_dataloaders(train_subset, validation_subset, test_subset, batch_size=32):
    train_dataset = CustomDataset(train_subset, transform=data_transforms['train'])
    val_dataset = CustomDataset(validation_subset, transform=data_transforms['val'])
    test_dataset = CustomDataset(test_subset, transform=data_transforms['test'])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    return train_loader, val_loader, test_loader

In [ ]:
from torch.utils.data import DataLoader
train_loader, val_loader, test_loader = create_dataloaders(train_subset, validation_subset, test_subset)

In [ ]:
# Filter out any None values that might be returned due to image loading errors
train_dataset = [(image, label) for image, label in train_subset if image is not None]
validation_dataset = [(image, label) for image, label in validation_subset if image is not None]
test_dataset = [(image, label) for image, label in test_subset if image is not None]

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm
import timm

model = timm.create_model('resnet18', pretrained=False)
print(model)
model.fc = nn.Linear(model.fc.in_features, len(label_encoder.classes_))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

def train_model(model, train_loader, validation_loader, criterion, optimizer, num_epochs=1):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {epoch_loss:.4f}")

        model.eval()
        validation_loss = 0.0
        corrects = 0
        with torch.no_grad():
            for images, labels in validation_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                validation_loss += loss.item() * images.size(0)

                _, preds = torch.max(outputs, 1)
                corrects += torch.sum(preds == labels.data)

        validation_loss = validation_loss / len(validation_loader.dataset)
        accuracy = corrects.double() / len(validation_loader.dataset)
        print(f"Validation Loss: {validation_loss:.4f}, Accuracy: {accuracy:.4f}")

train_loader, val_loader, test_loader = create_dataloaders(train_subset, validation_subset, test_subset)

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

In [ ]:
torch.save(model.state_dict(), 'model.pt')
print("Model saved to model.pt")

In [ ]:
import torch
from tqdm import tqdm

def test_model(model, test_loader):
    model.eval()
    corrects = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = corrects.double() / total
    print(f"Test Accuracy: {accuracy:.4f}")

    return all_preds, all_labels

all_preds, all_labels = test_model(model, test_loader)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, auc, roc_auc_score, accuracy_score
import seaborn as sns
import numpy as np
import torch
from tqdm import tqdm

# Assuming all_labels and all_preds are numpy arrays
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Ensure the model is in evaluation mode
model.eval()

# Get the probability scores for the positive class
all_probs = []
with torch.no_grad():
    for images, _ in tqdm(test_loader):
        images = images.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        all_probs.extend(probs.cpu().numpy())

all_probs = np.array(all_probs)

# Calculate evaluation metrics
cm = confusion_matrix(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)
roc_auc_macro = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='macro')
roc_auc_weighted = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='weighted')

# Plot and save confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_matrix.png')
plt.close()

# Calculate ROC curve and AUC for each class
n_classes = len(label_encoder.classes_)
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(all_labels == i, all_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot and save ROC curve
plt.figure()
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.savefig('roc_curve.png')
plt.close()

# Write report to text file
with open('evaluation_report.txt', 'w') as f:
    f.write(f"Evaluation Accuracy: {accuracy:.4f}\n")
    f.write(f"F1 Score: {f1:.4f}\n")
    f.write(f"Macro AUC: {roc_auc_macro:.4f}\n")
    f.write(f"Weighted AUC: {roc_auc_weighted:.4f}\n")
    f.write("\nClass-wise AUC Scores:\n")
    for i in range(n_classes):
        f.write(f"Class {i} AUC: {roc_auc[i]:.4f}\n")

print("Evaluation report saved to evaluation_report.txt")
print("Confusion matrix image saved to confusion_matrix.png")
print("ROC curve image saved to roc_curve.png")
